In [1]:
# pip install filterpy, https://buildmedia.readthedocs.org/media/pdf/filterpy/latest/filterpy.pdf
import filterpy as fp
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pickle as pkl

from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.kalman import UnscentedKalmanFilter as UKF

In [2]:
# Dynamics and Measurement Model Definition
nx = 5
nz = 3
dt = 0.01

def normalize_angle(ang):
    while ang >= 2 * np.pi:
        ang -= 2 * np.pi
    while ang <= - 2 * np.pi:
        ang += 2 * np.pi
    assert(ang >= -np.pi and ang <= np.pi)
    return ang

def f_cv(z, dt):
    z_new = np.zeros(nx)
    z_new[0] = z[0] + z[3] * np.cos(z[2]) * dt  #x
    z_new[1] = z[1] + z[3] * np.sin(z[2]) * dt  #y
    z_new[2] = z[2] + z[4] * dt                 #theta
    z_new[3] = z[3]                             #v
    z_new[4] = z[4]                             #omega
    return z_new
        
def h_cv(z):
    return [z[0], z[1], normalize_angle(z[2])] 

In [3]:
# Modified from https://github.com/rlabbe/filterpy/blob/master/filterpy/kalman/UKF.py
def state_mean(sigmas, Wm):
    assert(sigmas.shape[1] == nx)
    x = np.zeros(nx)

    sum_sin = np.sum(np.dot(np.sin(sigmas[:, 2]), Wm))
    sum_cos = np.sum(np.dot(np.cos(sigmas[:, 2]), Wm))
    for i in range(nx):
        if i is 2:
            x[i] = np.atan2(sum_sin, sum_cos)       # angle weighted avg
        else:
            x[i] = np.sum(np.dot(sigmas[:, i], Wm)) # normal weighted avg
    return x

def z_mean(sigmas, Wm):
    assert(sigmas.shape[1] == nz)
    z = np.zeros(nz)

    sum_sin = np.sum(np.dot(np.sin(sigmas[:, 2]), Wm))
    sum_cos = np.sum(np.dot(np.cos(sigmas[:, 2]), Wm))
    for i in range(nz):
        if i is 2:
            z[i] = np.atan2(sum_sin, sum_cos)
        else:
            z[i] = np.sum(np.dot(sigmas[:,i], Wm))
    return z

def residual_x_z(a, b):
    y = a - b
    y[2] = normalize_angle(y[2])
    return y

In [4]:
points = MerweScaledSigmaPoints(n=3, alpha=.00001, beta=2, kappa=0, 
                                    subtract=residual_x_z)
ukf = UKF(dim_x=nx, dim_z=nz, fx=f_cv, hx=h_cv,
          dt=dt, points=points, x_mean_fn=state_mean, 
          z_mean_fn=z_mean, residual_x=residual_x_z, 
          residual_z=residual_x_z)

# initial state and covariance estimate
ukf.x = np.array([0., 0., -np.pi/2., 1.0, 0.0])
ukf.P = np.eye(nx)

# process noise
ukf.Q = np.diag([1e-3, 1e-3, 1e-3, 1., 1.])

# measurement noise
ukf.R = np.diag([1e-3, 1e-3, 1e-3])


In [ ]:
# TODO: load in data from the rosbag
# initial position set based on spawn location
# just run this one step at a time and compare to real trajectory
# add prediction component
# IMM formulation